# 2. Facial Embeddings

Nowadays, to create a facial recognition model, we don't need to train our own model. Experts has helped us to pre-train a model, we just need to learn how to use it. This notebook will teach you how to use their deep learning models on your own dataset, try out multiple cutting-edge models, and compare their performances.

In [1]:
import cv2
import matplotlib.pyplot as plt
import keras_vggface as kv
import modules.utils as utils
from tensorflow.keras.models import save_model
import tensorflow as tf
from tensorflow.python.keras.engine import training
from tensorflow import keras
import os
from statistics import mean
import pandas as pd

## Face Preprocess

Here we will use the `FacePreprocess` module to clean and normalize the input image before input to the deep learning model.

In [2]:
from modules.FacePreprocess import FacePreprocess
ssd_model = r'./models/ssd/deploy.prototxt.txt'
ssd_weights = r'./models/ssd/res10_300x300_ssd_iter_140000.caffemodel'
processor = FacePreprocess(ssd_model, ssd_weights)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
img_path = './dataset/joy/img_1.jpg'
img = cv2.imread(img_path)
processed_img = processor.preproc(img)[0][0]

fig = plt.figure(figsize=(10, 7))
fig.add_subplot(1, 2, 1)  
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Original")
fig.add_subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Preprocessed")

Text(0.5, 1.0, 'Preprocessed')

## Compute Facial Embeddings

In this step, we will take our input image and insert them into the deep learning models. These models will compute embeddings of our input image, which we will use to compare with the embeddings of other images.

In [4]:
# stores list of models
models = {}

#### ⭐ keras-vggface (kv)
source: https://github.com/rcmalli/keras-vggface

\>\> resnet50

In [5]:
model = kv.VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
models['kv-resnet50'] = [model, (224,224)]
img_representation = model.predict(utils.resize(processed_img, (224, 224)))[0,:]

2023-09-14 21:06:46.798725: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-09-14 21:06:46.798747: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-14 21:06:46.798753: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-14 21:06:46.799153: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-14 21:06:46.799481: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-09-14 21:06:48.062240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 606ms/step


\>\> senet50

In [6]:
model = kv.VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
models['kv-senet50'] = [model, (224,224)]
img_representation = model.predict(utils.resize(processed_img, (224, 224)))[0,:]

2023-09-14 21:06:49.722063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 851ms/step


\>\> vgg16

In [7]:
model = kv.VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='avg')
models['kv-vgg16'] = [model, (224,224)]
img_representation = model.predict(utils.resize(processed_img, (224, 224)))[0,:]

1/1 [==============================] - 0s 94ms/step


2023-09-14 21:06:50.553595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


#### ⭐ ArcFace
source: https://sefiks.com/2020/12/14/deep-face-recognition-with-arcface-in-keras-and-python/

In [8]:
model = keras.models.load_model('./models/arcface/model.keras', compile=False)
models['arcface'] = [model, (112,112)]
img_representation = model.predict(utils.resize(processed_img, (112, 112)))[0,:]

2023-09-14 21:06:51.898282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step


#### ⭐ Facebook DeepFace
source: https://sefiks.com/2020/02/17/face-recognition-with-facebook-deepface-in-keras/

In [9]:
model = keras.models.load_model('./models/deepface/model.keras', compile=False)
models['deepface'] = [model, (152,152)]
img_representation = model.predict(utils.resize(processed_img, (152, 152)))[0,:]

2023-09-14 21:07:05.494125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 51s 51s/step


#### ⭐ Google Facenet
source: https://sefiks.com/2018/09/03/face-recognition-with-facenet-in-keras/

In [10]:
model = keras.models.load_model('./models/facenet/model.keras', compile=False)
models['facenet'] = [model, (160,160)]
img_representation = model.predict(utils.resize(processed_img, (160, 160)))[0,:]

2023-09-14 21:07:51.787284: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step


## Compare Embeddings of the Images

The general idea is to compare the vector distance of the images, using the embeddings from the models. In theory, two images of the same person should have a closer distance than two images of different people. Here I use cosine distance and euclidean distance to compare, but other vector distance calculation methods can be used too.


\>\> `./dataset/joy/img_1` vs `./dataset/joy/img_2`

In [11]:
img2_path = './dataset/joy/img_2.jpg'
processed_img2 = processor.preproc(cv2.imread(img2_path))[0][0]

table = []
for key, value in models.items():
    img1_r = value[0].predict(utils.resize(processed_img, value[1]), verbose=False)[0,:]
    img2_r = value[0].predict(utils.resize(processed_img2, value[1]), verbose=False)[0,:]

    table.append([key, utils.findCosineDistance(img1_r, img2_r), utils.findEuclideanDistance(utils.L2_normalize(img1_r), utils.L2_normalize(img2_r))])
pd.DataFrame(table, columns=['model name', 'cosine distance', 'euclidean distance'])

model name  cosine distance  euclidean distance
0  kv-resnet50         0.250433            0.707718
1   kv-senet50         0.015106            0.173817
2     kv-vgg16         0.105327            0.458972
3      arcface         0.059589            0.345222
4     deepface         0.283028            0.752368
5      facenet         0.056417            0.335906

\>\> `./dataset/joy/img_1` vs `./dataset/irene/img_2`

In [12]:
img2_path = './dataset/irene/img_2.jpg'
processed_img2 = processor.preproc(cv2.imread(img2_path))[0][0]

table = []
for key, value in models.items():
    img1_r = value[0].predict(utils.resize(processed_img, value[1]), verbose=False)[0,:]
    img2_r = value[0].predict(utils.resize(processed_img2, value[1]), verbose=False)[0,:]

    table.append([key, utils.findCosineDistance(img1_r, img2_r), utils.findEuclideanDistance(utils.L2_normalize(img1_r), utils.L2_normalize(img2_r))])
pd.DataFrame(table, columns=['model name', 'cosine distance', 'euclidean distance'])

model name  cosine distance  euclidean distance
0  kv-resnet50         0.504777            1.004765
1   kv-senet50         0.008683            0.131783
2     kv-vgg16         0.157484            0.561221
3      arcface         0.057538            0.339229
4     deepface         0.537266            1.036596
5      facenet         0.037936            0.275449

We can see that the distances for `joy/img_1 vs joy/img_2` is closer than `joy/img_1 vs irene/img_2` (except for `kv-senet50`, `arcface`, `facenet`). The closer the distance, the more similar the person in both pictures are. For the models which predicted wrongly, it doesn't mean that the model is bad, it simply means that the model doesn't suit our dataset. Not one model is better than the other, some models just suit our dataset better than others. A lot of trial and error is needed to find the most suitable one for your dataset!

## Compare Models

In this part, we want to compare the performance of our models, by computing the distance between `./dataset/joy/img_1` and every other image in the dataset. The printed outputs are the average distance between `./dataset/joy/img_1` and the images in each folder.

#### ⭐ cosine distance

&ensp;&ensp;&ensp; $\ d(\mathbf{p}, \mathbf{q}) = 1 - \frac{p^\top q}{\sqrt{p^\top p} \sqrt{q^\top q}} $

In [13]:
table = []
for key, value in models.items():
    img1_r = value[0].predict(utils.resize(processed_img, value[1]), verbose=False)[0,:]

    header = ['model name']
    avg = [key]
    for id in os.listdir('./dataset'):
        dist = []
        header.append(id)
        for i in range(1,6):
            if id == 'joy' and i == 1: # skip because distance of the same image will be 0
                continue

            processed_img2 = processor.preproc(cv2.imread('./dataset/{}/img_{}.jpg'.format(id, str(i))))[0][0]
            img2_r = value[0].predict(utils.resize(processed_img2, value[1]), verbose=False)[0,:]
            dist.append(utils.findCosineDistance(img1_r, img2_r))
        avg.append(mean(dist))
    table.append(avg)

cos_table = pd.DataFrame(table, columns=header)
cos_table

model name      yeri     irene       joy    seulgi     wendy
0  kv-resnet50  0.371814  0.469357  0.256579  0.514212  0.457680
1   kv-senet50  0.016033  0.010082  0.011687  0.011988  0.017053
2     kv-vgg16  0.136590  0.151111  0.128576  0.156376  0.227465
3      arcface  0.070131  0.056845  0.048536  0.067075  0.065084
4     deepface  0.383167  0.485320  0.508817  0.530104  0.526895
5      facenet  0.052261  0.042866  0.039898  0.028467  0.052234

The distance for `joy` should be lower than the distance in the other columns. All the models (except `kv-senet50`, `facenet`, `deepface`) performs as expected.

#### ⭐ euclidean distance

&ensp;&ensp;&ensp; L<sup>2</sup> norm &rarr; $\ ||x||_2 = \sqrt{\sum_{i=1}^{n} x_i} $

&ensp;&ensp;&ensp; Euclidean dist. &rarr; $\ d(\mathbf{p}, \mathbf{q}) = \sqrt{\sum_{i=1}^{n} (||q_i||_2 - ||p_i||_2)^2} $

In [14]:
table = []
for key, value in models.items():
    img1_r = value[0].predict(utils.resize(processed_img, value[1]), verbose=False)[0,:]

    header = ['model name']
    avg = [key]
    for id in os.listdir('./dataset'):
        dist = []
        header.append(id)
        for i in range(1,6):
            if id == 'joy' and i == 1: # skip because distance of the same image will be 0
                continue

            processed_img2 = processor.preproc(cv2.imread('./dataset/{}/img_{}.jpg'.format(id, str(i))))[0][0]
            img2_r = value[0].predict(utils.resize(processed_img2, value[1]), verbose=False)[0,:]
            dist.append(utils.findEuclideanDistance(utils.L2_normalize(img1_r), utils.L2_normalize(img2_r)))
        avg.append(mean(dist))
    table.append(avg)

eucl_table = pd.DataFrame(table, columns=header)
eucl_table

model name      yeri     irene       joy    seulgi     wendy
0  kv-resnet50  0.862289  0.968208  0.712170  1.013652  0.956041
1   kv-senet50  0.175030  0.138873  0.151743  0.154132  0.182781
2     kv-vgg16  0.522085  0.549266  0.505542  0.558935  0.673387
3      arcface  0.357699  0.330638  0.309097  0.360444  0.358644
4     deepface  0.851106  0.969131  0.998021  1.026721  1.020671
5      facenet  0.307432  0.286378  0.280053  0.235872  0.308538

The distance for `joy` should be lower than the distance in the other columns. All the models (except `kv-senet50`, `facenet`, `deepface`) performs as expected.

Between the rest of the models, how to pick the most suitable model? We want to pick the model in which the _'wrong class'_ has the furthest distance from the _'correct class'_. Below is a simple code to check (there are better ways to write the code but since our dataset is small this will do)

In [15]:
table = []
for i, row in cos_table.iterrows():
    tmp = [row['model name']]

    # cosine
    tmp.append(mean([row['irene'], row['seulgi'], row['wendy'], row['yeri']]))
    # euclid
    tmp.append(mean([eucl_table['irene'][i], eucl_table['seulgi'][i], eucl_table['wendy'][i], eucl_table['yeri'][i]]))
    
    table.append(tmp)

df = pd.DataFrame(table, columns=['model name', 'cosine', 'euclidean'])
df.sort_values(['cosine', 'euclidean'], ascending=False)

model name    cosine  euclidean
4     deepface  0.481372   0.966907
0  kv-resnet50  0.453266   0.950048
2     kv-vgg16  0.167885   0.575918
3      arcface  0.064784   0.351856
5      facenet  0.043957   0.284555
1   kv-senet50  0.013789   0.162704

The table above shows the average distance of the _'wrong class'_. As we can see, the top three models for our dataset are deepface, kv-vgg16, and kv-senet50. Keep in mind that these numbers are specific to this dataset only. Your dataset might have different results.

## Conclusion

This notebook's purpose is to try out different cutting-edge machine learning models, and it works well with small datasets. However, with larger datasets, we need a better method to compare distances. Check out the next notebook!